Import Required Libraries

In [1]:
%pip install word2number spacy python-Levenshtein
!wget https://raw.githubusercontent.com/Valspir/Meal_Planner/main/recipes.db

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
--2023-04-09 18:25:21--  https://raw.githubusercontent.com/Valspir/Meal_Planner/main/recipes.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3977216 (3.8M) [application/octet-stream]
Saving to: ‘recipes.db.2’

recipes.db.2        100%[===================>]   3.79M  1.78MB/s    in 2.1s    

2023-04-09 18:25:23 (1.78 MB/s) - ‘recipes.db.2’ saved [3977216/3977216]



In [25]:
import spacy
import sqlite3
import json
import pprint
from word2number import w2n
from fractions import Fraction
import re
from Levenshtein import distance
import requests
import time


Define functions


In [68]:
measurementArr = ["tsp", "teaspoon", "tbsp", "tablespoon", "cup", "pint", "quart", "gallon", "oz", "ounce", "lb", "pound", "g", "gram", "mg", "pieces","cm", "ml", "l", "pkt", "kg", "can"]
blacklistWords = ["notes", "diagonally", "woolworths"]
whitelistWords = ["cornflour","oregano","milk", "tomato", "rice", "cream", "cider", "parsley", "avocado", "leek", "carrot"]
meatArr = ['lamb', 'beef', 'chicken', 'pork', 'turkey', 'salmon', 'barramundi', 'ham', 'prawn', 'fish', 'bacon']

def convert_to_float(quantity_str):
    # Check if quantity_str contains a forward slash
    if '/' in quantity_str:
        # Split the string into two parts, before and after the forward slash
        parts = quantity_str.split('/')
        # Convert the first part to an integer and the second part to a fraction
        quantity = int(parts[0]) + Fraction(parts[1])
        # Convert the fraction to a float and return the result
        return float(quantity)
    
    try:
        # Convert quantity string to float directly
        return float(quantity_str)
    except ValueError:
        pass
    
    # Convert words to numbers
    words = quantity_str.split()
    for i, word in enumerate(words):
        if word.isdigit():
            continue
        try:
            num = w2n.word_to_num(word)
            words[i] = str(num)
        except ValueError:
            pass
    
    # Convert fractions to decimals
    quantity_str = " ".join(words)
    quantity = sum(Fraction(s) for s in quantity_str.split() if '/' in s)
    quantity += sum(float(s) for s in quantity_str.split() if '/' not in s)
    
    return float(quantity)

    
    return quantity


def preprocess_recipe_text(recipe_text):
    recipe_text = recipe_text.strip().lower()
    #recipe_text = recipe_text.replace("\n", " ")
    recipe_text = recipe_text.split(",")[0]
    recipe_text = re.sub("\(([^\)]+)\)","",recipe_text)
    return recipe_text

def getMeat(text):
    for meat in meatArr:
        if text == meat:
            return meat

def extract_ingredient_quantity(recipe_data):
    ingredient_quantity = {}
    recipeMeats = []
    recipeIngredients = []
    for data in recipe_data:
        data = preprocess_recipe_text(data)
        doc = nlp(data)
        quantity = "0"
        posIngredients = []
        children = []
        ingredient = ""
        measurement = ""
        for token in doc:
            if str(token.text).lower() in meatArr:
                recipeMeats.append(str(token.text))
            #print("--"*10)
            #print(token.text)
            #print(token.ent_type_)
            #print(token.pos_)
            tokenInfo = {}
            tokenInfo["ingredientStr"] = str(doc)
            if token.pos_ == "QUANTITY" or token.pos_ == "NUM":
                try:
                    quantity = str(convert_to_float(token.text))
                except:
                    pass
            elif str(re.sub("\d","",token.text)) in measurementArr:
                measurement = str(re.sub("\d","",str(token.text)))
                qty = str(convert_to_float(str(re.sub(measurement,"",token.text))))
                if float(qty) > 0.0:
                    quantity = qty
            elif token.text in measurementArr:
                measurement = token.text
            elif (str(token.pos_) == "NOUN" or str(token.pos_) == "PROPN" or str(token.pos_) == "ADJ" or str(token.ent_type_) == "FOOD" or str(token.text) in whitelistWords) and re.findall("\d",str(token.text)) == [] and str(token.text) not in blacklistWords:
                tokenInfo["ingredient"] = token.text
                children = []
                foundNext = 1
                nextToken = token
                while(foundNext):
                  try:#I could replace most of the code with this try block if I move the quantity and measurement code into the try block
                    nextToken = nextToken.nbor()
                    child = nextToken
                    if((str(child.pos_) not in ["ADV", "VERB", "PUNCT", "SCONJ", "NUM"]) and (str(child.ent_type_) not in ["QUANTITY","CARNDINAL"]) and (str(child.text) not in measurementArr) and (str(child.text) not in blacklistWords) and (str(re.sub("\d","",child.text)) not in measurementArr)) or str(child.text) in whitelistWords:
                        children.append(str(child.text))
                  except:
                    foundNext = 0
            tokenInfo["children"] = children
            if "ingredient" in list(tokenInfo.keys()):
               posIngredients.append(tokenInfo)
            bestMatch = []
            bestCount = -1
        for possible in posIngredients:
            if len(possible["children"]) > bestCount:
                bestMatch = possible
                bestCount = len(possible["children"])
                #print(possible)
        ing = ""
        if bestMatch == []:
            return -1
            '''print(doc)
            for token in doc:
                print("--"*10)
                print("+"+token.text+"+")
                print(token.ent_type_)
                print(token.pos_)
            i = input("Continue? ")
            if i == "n":
                exit()'''
        if(bestMatch != []):
            meat = 0
            for child in bestMatch["children"]:
                #print("Checking for meat in: "+child)
                if child in meatArr:
                    #print("Found meat in: "+child)
                    meat = getMeat(child)
                    #recipeMeats.append(meat)
                ing+=" "+child
            #print(bestMatch)
            recipeIngredients.append([quantity,measurement,str(bestMatch["ingredient"]+ing),str(doc)])
            ingredient_quantity[bestMatch["ingredient"]+ing] = bestMatch
            if meat:
              ingredient_quantity[bestMatch["ingredient"]+ing]['Meat'] = meat
    if recipeMeats == []:
        recipeMeats = ["None"]
    return [recipeMeats, recipeIngredients]

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except:
        print("E")

    return conn


url = 'https://www.woolworths.com.au/apis/ui/Search/products'


def getProdPrice(prod, qty, measurement):
  if(len(prod) > 100):
      return -1

  d = {
      "EnableGp": "false",
      "ExcludeSearchTypes": ["UntraceableVendors"],
      "Filters": [],
      "IsSpecial": "false",
      "Location": "/shop/search/products?searchTerm="+prod,
      "PageNumber": 1,
      "PageSize": 36,
      "SearchTerm": prod,
      "SortType": "TraderRelevance"
  }

  h = {
      'Content-Type' : 'application/json',
      'origin'       : 'https://www.woolworths.com.au',
      'referer'      : 'https://www.woolworths.com.au/shop/search/products?searchTerm=red%20onion',
      'user-agent'   : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
      'Accept'       : 'application/json, text/plain, */*',
      'Accept-Language': 'EN-US,en;q-0.5'
  }

  session = requests.Session()

  r = session.get("https://www.woolworths.com.au/shop/search/products?searchTerm=red%20onion", headers=h, cookies=session.cookies)
  #print(r.content)
  h = {
      'Content-Type' : 'application/json',
      'origin'       : 'https://www.woolworths.com.au',
      'referer'      : 'https://www.woolworths.com.au/shop/search/products?searchTerm=red%20onion',
      'user-agent'   : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
      'Accept'       : 'application/json, text/plain, */*',
      'Accept-Language': 'EN-US,en;q-0.5'
  }





  r = session.post(url, headers=h, json=d, cookies=session.cookies)
  while True:
      r = session.post(url, headers=h, json=d, cookies=session.cookies)
      jsonData = json.loads(r.text)
      if len(jsonData) > 1:
          products = jsonData["Products"]
          SuggestedTerm = jsonData["SuggestedTerm"]
          break
      print("API Error")
      time.sleep(10)

  baseProduct = prod
  bestMatch = []
  cheapest = [10000,1000,[]]
  nearest = [10000,1000,[]]
  allProds = []

  if products != None:
    for product in products:
      if product["Products"][0]["Price"]:
        foundPrice = float(product["Products"][0]["Price"])
        foundProduct = extract_ingredient_quantity([product["DisplayName"]])
        if foundProduct != -1:
          #print(str(foundProduct)+": $"+str(foundPrice))
          if measurement != "" and measurement not in ["tbsp", "tsp", "cup"]:
            if foundProduct[1][0][1] == measurement:
              dist = float(foundProduct[1][0][0])-float(qty)
              if dist > -1:
                if dist < nearest[1]:
                  nearest = [foundPrice, dist, foundProduct[1][0]]
                  #print("New Nearest: " + str(nearest))
                if foundPrice < cheapest[0]:
                  cheapest = [foundPrice, dist, foundProduct[1][0]]
                  #print("New Cheapest: " + str(cheapest))
                  #bestMatch = foundProduct
          else:
            #print(str(foundProduct)+": $"+str(foundPrice))
            allProds.append([foundPrice,foundProduct[1][0]])

  avgPrice = 0
  if allProds != []:
    aP = sorted(allProds)
    aPL = len(aP)
    if(aPL != 1):
      if float(aPL/2).is_integer():
        avgPrice = aP[int(aPL/2)][0]
      else:
        #try:
          #print(aP[int((aPL/2)+0.5)])
        #except:
          #input("Stopped")
        price1 = aP[int((aPL/2)-0.5)][0]
        price2 = aP[int((aPL/2)+0.5)][0]
        avgPrice = (price1+price2)/2
    else:
      avgPrice = aP[0][0]
  else:
    if nearest[0] != 10000 and cheapest[1] != 1000:
      avgPrice = (nearest[0]+cheapest[0])/2
    else:
      avgPrice = -1

  return avgPrice

allRecipes = []
shoppingList = []
ingredientArr = []
recipeData = []
ingredientID = 0
costArr = []
ingArr = []
lookupArr = []

In [ ]:
from IPython.display import clear_output
nlp = spacy.load('en_core_web_sm')

recipeDBConn = create_connection('recipes.db')
recipeCur = recipeDBConn.cursor()

pdConn = create_connection('processedData.db')
pdCur = pdConn.cursor()
sql = "CREATE TABLE IF NOT EXISTS Ingredients (recipeID int, ingredientID int)"
res = pdCur.execute(sql)
sql = "CREATE TABLE IF NOT EXISTS Cost (ingredientID int, cost float)"
res = pdCur.execute(sql)
sql = "CREATE TABLE IF NOT EXISTS Recipes (recipeID int, recipeName varchar, meat varchar, prepTime int, cookTime int, cost float, serves int, saves int, steps varchar, rating float)"
res = pdCur.execute(sql)

sql = 'DROP TABLE AltText'
res = pdCur.execute(sql)

sql = "CREATE TABLE IF NOT EXISTS AltText (ingredientID int, procText varchar, altText varchar)"
res = pdCur.execute(sql)

sql = "SELECT * FROM Recipes WHERE id>0"
res = recipeCur.execute(sql)
j = res.fetchall()

for r in j:
    clear_output(wait=True)
    id = r[0]
    recName = r[1]
    prepTime = r[7]
    cookTime = r[8]
    saves = r[9]
    serves = r[6]
    stepList = json.loads(r[5])
    stepCount = r[4]
    thisRecipe = []
    ingredientList = json.loads(r[3])
    for ingredient in ingredientList:
        thisRecipe.append(ingredient)
        allRecipes.append(ingredient)
    

    ingredient_quantity = extract_ingredient_quantity(thisRecipe)
    if(ingredient_quantity == -1):
        print(recName + " failed")
        continue

    thisRecipe = []
    print("ID: "+str(id)+"\nRecipe: "+str(recName)+"\nPrep Time: "+str(prepTime)+"\nCook Time: "+str(cookTime)+"\nServes: "+str(serves)+"\nMeat: "+ingredient_quantity[0][0])
    print("Ingredients: ")
    
    recipeCost = 0
    for ingredient in ingredient_quantity[1]:
    
        p = 0
        fiA = 0
        for ing in ingredientArr:
            #print(ing)
            i = ing[2]
            if((ingredient[2] == i or distance(ingredient[2],i) < 3) and ingredient[1] == ing[1]) and not fiA:
                fiA = 1
                #print(ingredientArr[p])
                ingredientArr[p][0] = float(ingredientArr[p][0])+float(ingredient[0])
                ingredientArr[p][3] += ","+ingredient[3]
                iid = -1
                for ingr in lookupArr:
                  if ingr[0] == i:
                    iid = ingr[1]
                sql = "INSERT INTO AltText(ingredientID,procText,altText) VALUES (?,?,?)"
                pdCur.execute(sql,(iid,str(ingredient[2]),str(ingredient[3])))  
            p+=1
        if not fiA:
            ingredientArr.append(ingredient)
            inArr = 0
            lookupArr.append([ingredient[2],ingredientID])
            #sql = "INSERT INTO Ingredients(recipeID, ingredientID) VALUES(?,?)"
            #pdCur.execute(sql,(id,ingredientID))
            

            ingCost = -1#float(getProdPrice(ingredient[2],ingredient[0],ingredient[1]))
            if ingCost == -1:
              print("   -"+str(ingredient[2])+": N/A")
            else:
              print("   -"+str(ingredient[2])+": $"+str(round(ingCost)))
              recipeCost+=ingCost
            sql = "INSERT INTO AltText(ingredientID,procText,altText) VALUES (?,?,?)"
            pdCur.execute(sql,(ingredientID,str(ingredient[2]),str(ingredient[3])))
            ingredientID+=1


    #print(shoppingList) #Format: [Item, Alt Text] (Alt text needed when parsing the ingredients fails)
    #print(recipeCost)

    #sql = "INSERT INTO Recipes(recipeID,recipeName,meat,prepTime,cookTime,cost,serves,saves,steps,rating) VALUES (?,?,?,?,?,?,?,?,?,?)"
    #pdCur.execute(sql,(id,recName,str(ingredient_quantity[0][0]),prepTime,cookTime,recipeCost,serves,saves,r[5],0))
    pdConn.commit()
    print("Estimated Cost: $"+str(round(recipeCost)))
    print(str(stepCount) + " Steps")
    for step in stepList:
      print("   -"+step)
    print()
    recipeData.append([[id,recName],[id,"ingredient_id"],[id,round(recipeCost),prepTime,cookTime,stepCount]])
    print("---"*30+"\n")
for i in ingredientArr:
    if(i[1] not in ["tbsp", "tsp", "cup"] and float(i[0]) > 0):
        if(i[1] == ""):
            shoppingList.append([str(round(float(i[0])))+" x "+str(i[2]), str(i[3])])
        else:
            if(i[1] in ["ml", "l", "g", "mg"]):
                shoppingList.append([str(round(float(i[0])))+str(i[1])+" "+str(i[2]), str(i[3])])
            else:
                shoppingList.append([str(round(float(i[0])))+" "+str(i[1])+" "+str(i[2]), str(i[3])])
    else:
        shoppingList.append([str(i[2]), str(i[3])])

for i in shoppingList:
    print(str(i))

pprint.pprint(ingArr)
pprint.pprint(lookupArr)
pprint.pprint(costArr)

In [72]:
recipeDBConn = create_connection('recipes.db')
recipeCur = recipeDBConn.cursor()
sql = 'SELECT id,recipeName,ingredients,steps FROM Recipes WHERE id=3;'
res = recipeCur.execute(sql)
j = res.fetchall()
for recipe in j:
    print("ID: "+ str(recipe[0]))
    print("Name: " + recipe[1])
    stepList = json.loads(recipe[2])
    for step in stepList:
      print("   -"+step)


ID: 3
Name: Ham and cheese slab pie
   -41⁄4 sheets frozen butter puff pastry, just thawed
   -250g (about 7 slices) double smoked leg ham, trimmed
   -180g (1⁄2 cup) bought caramelised onion relish
   -1 egg, lightly whisked
   -100g butter, chopped
   -1 garlic clove, crushed
   -50g (1⁄3 cup) plain flour
   -500ml (2 cups) milk
   -1 tbsp dijon mustard
   -200g (2 cups) grated vintage cheddar


Woolies testing


In [ ]:
import requests
import pprint
import json

url = 'https://www.woolworths.com.au/apis/ui/Search/products'
#url = "https://httpbin.org/post"

def getProdPrice(prod):

  d = {
      "EnableGp": "false",
      "ExcludeSearchTypes": ["UntraceableVendors"],
      "Filters": [],
      "IsSpecial": "false",
      "Location": "/shop/search/products?searchTerm="+prod,
      "PageNumber": 1,
      "PageSize": 36,
      "SearchTerm": prod,
      "SortType": "TraderRelevance"
  }

  h = {
      'Content-Type' : 'application/json',
      'origin'       : 'https://www.woolworths.com.au',
      'referer'      : 'https://www.woolworths.com.au/shop/search/products?searchTerm=red%20onion',
      'user-agent'   : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
      'Accept'       : 'application/json, text/plain, */*',
      'Accept-Language': 'EN-US,en;q-0.5'
  }

  session = requests.Session()

  r = session.get("https://www.woolworths.com.au/shop/search/products?searchTerm=red%20onion", headers=h, cookies=session.cookies)
  #print(r.content)
  h = {
      'Content-Type' : 'application/json',
      'origin'       : 'https://www.woolworths.com.au',
      'referer'      : 'https://www.woolworths.com.au/shop/search/products?searchTerm=red%20onion',
      'user-agent'   : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
      'Accept'       : 'application/json, text/plain, */*',
      'Accept-Language': 'EN-US,en;q-0.5'
  }





  r = session.post(url, headers=h, json=d, cookies=session.cookies)
  products = json.loads(r.text)["Products"]


  baseProduct = extract_ingredient_quantity([prod])
  bestMatch = []
  cheapest = [10000,1000,[]]
  nearest = [10000,1000,[]]

  for product in products:
    foundPrice = float(product["Products"][0]["Price"])
    foundProduct = extract_ingredient_quantity([product["DisplayName"]])
    if foundProduct[1][0][1] == baseProduct[1][0][1]:
      dist = float(foundProduct[1][0][0])-float(baseProduct[1][0][0])
      if dist > -1:
        if dist < nearest[1]:
          nearest = [foundPrice, dist, foundProduct[1][0]]
        if foundPrice < cheapest[0]:
          cheapest = [foundPrice, dist, foundProduct[1][0]]
          #bestMatch = foundProduct

  '''print(nearest)
  print(cheapest)'''
  return (nearest[0]+cheapest[0])/2

In [ ]:
recipeDBConn = create_connection('recipes.db')
recipeCur = recipeDBConn.cursor()
#sql = 'SELECT id,recipeName,prepTime,cookTime,servings,saves FROM Recipes WHERE prepTime < 30 AND cookTime < 30 and cookTime != -1 ORDER BY saves DESC'
sql = 'SELECT Count(id) FROM Recipes'
res = recipeCur.execute(sql)
j = res.fetchall()
pprint.pprint(j)

TODO:



-Insert data into relational db
```
  -What data do I want?
    -Ingredients
      -Ingredient ID
      -Processed Text
      -Raw Text
      -Rough Ingredient Cost
    -Main
      -id
      -Recipe Name
    -Ingredient Linking
      -id
      -Ingredient ID
    -Attributes
      -id
      -Rough Meal Cost
      -Prep Time
      -Cook Time
      -Step Count
```

-Recommend recipes based on ingredients
